# Keener and Sneyd Blood Model


In [1]:
import numpy as np
from bokeh.plotting import ColumnDataSource, figure, output_notebook, show
from bokeh.layouts import row, widgetbox, gridplot
from bokeh.models import CustomJS, Slider, Range1d, LabelSet, Label, Span
import warnings

warnings.simplefilter('ignore')


def saturationFxn(oPP, kp, expTerm):
    percentSaturation = ( (oPP**expTerm) / ((kp**expTerm)+(oPP**expTerm)) ) * 100
    return percentSaturation

oPP = np.arange(0,120,1)

kp_mb = 2.6
n_mb = 1
myoglobinPercentSaturation  = saturationFxn(oPP, kp_mb, n_mb)

kp_hb = 26
n_hb = 2.5
hemoglobinPercentSaturation = saturationFxn(oPP, kp_hb, n_hb)



o_ppValArray = 100*np.ones(len(oPP))
myoglobinPS_at_opp = np.linspace(0,[saturationFxn(o_ppValArray[1], kp_mb, n_mb)],len(oPP))
hemoglobinPS_at_opp = np.linspace(0,[saturationFxn(o_ppValArray[1], kp_hb, n_hb)],len(oPP))

# I want an array that goes from 0 to oPPval, in the same length as oPP

yLine_oppValArray = np.linspace(0,o_ppValArray[1],len(oPP))
yLine_myoglobinPS = myoglobinPS_at_opp[-1]*np.ones(len(oPP))
yLine_hemoglobinPS = hemoglobinPS_at_opp[-1]*np.ones(len(oPP))

source = ColumnDataSource(data=dict(oPP=oPP, 
                                    myoglobinPercentSaturation=myoglobinPercentSaturation, 
                                    hemoglobinPercentSaturation=hemoglobinPercentSaturation,       
                                    o_ppValArray = o_ppValArray,
                                    myoglobinPS_at_opp = myoglobinPS_at_opp,
                                    hemoglobinPS_at_opp = hemoglobinPS_at_opp,
                                    yLine_oppValArray = yLine_oppValArray,
                                    yLine_myoglobinPS = yLine_myoglobinPS,
                                    yLine_hemoglobinPS = yLine_hemoglobinPS,
                                   ))

TOOLTIPS = [
    ("a", "$myoglobinPercentSaturation"),
    ("b", "$hemoglobinPercentSaturation"),
    ("c", "$oPP"),
]

#genericCasePlot = figure(title = "Keener & Sneyd Blood Model",tools ='hover')
genericCasePlot = figure(title = "Keener & Sneyd Blood Model")

# make invisible points at minima and maxima to fix the plot
genericCasePlot.scatter(np.max(oPP),100,alpha=0)
genericCasePlot.scatter(0,0,alpha=0)
genericCasePlot.xaxis.axis_label = "Oxygen Partial Pressure (mm Hg)"
genericCasePlot.yaxis.axis_label = "Saturation (%)"

genericCasePlot.xaxis.axis_label_text_font_style = 'bold'
genericCasePlot.xaxis.axis_label_text_font_size = '20pt'

genericCasePlot.yaxis.axis_label_text_font_style = 'bold'
genericCasePlot.yaxis.axis_label_text_font_size = '20pt'
genericCasePlot.yaxis.major_label_text_font_style = 'bold'
genericCasePlot.yaxis.major_label_text_font_size = '10pt'

genericCasePlot.line('oPP','myoglobinPercentSaturation', source = source, line_width = 3,
                     line_alpha = 1, line_color = "red" , legend_label = 'Mb')
genericCasePlot.line('oPP','hemoglobinPercentSaturation', source = source, line_width = 3,
                     line_alpha = 1, line_color = "blue", legend_label = 'Hb')

genericCasePlot.line('o_ppValArray','myoglobinPS_at_opp', source = source, line_width = 2,
                     line_alpha = 1, line_color = "red", line_dash = "dashed")
genericCasePlot.line('o_ppValArray','hemoglobinPS_at_opp', source = source, line_width = 2,
                     line_alpha = 1, line_color = "blue", line_dash = "dotted")

genericCasePlot.line('yLine_oppValArray','yLine_myoglobinPS', source = source, line_width = 2,
                     line_alpha = 1, line_color = "red", line_dash = "dashed")
genericCasePlot.line('yLine_oppValArray','yLine_hemoglobinPS', source = source, line_width = 2,
                     line_alpha = 1, line_color = "blue", line_dash = "dotted")

kp_mb_slider = Slider(start=0, end= 100, value= 2.6, step= 0.1, title="K_Mb")
kp_hb_slider = Slider(start=0, end= 100, value= 26, step= 0.1, title="K_Hb")

#n_mb_slider = Slider(start=0, end= 50, value= 1, step= 0.1,title="n_Mb")
n_hb_slider = Slider(start=0, end= 20, value= 2.5, step= 0.1,title="n")

o_pp_fig_slider = Slider(start=np.min(oPP), end= np.max(oPP), value= 100, step= 0.1,
                         title="Oxygen Partial Pressure Data Cursor")


yLine_oppValArray = np.linspace(0,o_ppValArray[1],len(oPP))
yLine_myoglobinPS = myoglobinPS_at_opp[-1]*np.ones(len(oPP))
yLine_hemoglobinPS = hemoglobinPS_at_opp[-1]*np.ones(len(oPP))


callback = CustomJS(args=dict(source=source, kp_mb_ = kp_mb_slider, kp_hb_ = kp_hb_slider, 
                              n_hb_ = n_hb_slider, o_ppVal_ = o_pp_fig_slider), 
                    code="""
    
    var data = source.data;
    var oPP = data['oPP']
    var myoglobinPercentSaturation = data['myoglobinPercentSaturation']
    var hemoglobinPercentSaturation = data['hemoglobinPercentSaturation']
    var kp_mb = kp_mb_.value
    var kp_hb = kp_hb_.value
    var n_hb = n_hb_.value
    var o_ppValArray = data['o_ppValArray'];
    var myoglobinPS_at_opp = data['myoglobinPS_at_opp'];
    var hemoglobinPS_at_opp = data['hemoglobinPS_at_opp'];
    
    var yLine_oppValArray = data['yLine_oppValArray'];
    var yLine_myoglobinPS = data['yLine_myoglobinPS'];
    var yLine_hemoglobinPS = data['yLine_hemoglobinPS'];
    
    var o_ppVal = o_ppVal_.value;
    
    var myoglobinPS_at_oppVal  = (( (o_ppVal ** 1   ) / ((kp_mb ** 1) + (o_ppVal ** 1   )) ) ) * 100;
    var hemoglobinPS_at_oppVal = (( (o_ppVal** n_hb) / ((kp_hb** n_hb)+ (o_ppVal)** n_hb) ) ) * 100;


    
    var myoglobinPS_StepVal = myoglobinPS_at_oppVal  / myoglobinPS_at_opp.length
    var hemoglobinPS_StepVal = hemoglobinPS_at_oppVal  / hemoglobinPS_at_opp.length
    
    o_ppValArray.fill(o_ppVal)
    myoglobinPS_at_opp.fill(0)
    hemoglobinPS_at_opp.fill(0)
    
    myoglobinPS_at_opp[myoglobinPS_at_opp.length-1] = myoglobinPS_at_oppVal
    hemoglobinPS_at_opp[hemoglobinPS_at_opp.length-1] = hemoglobinPS_at_oppVal
 
    yLine_oppValArray.fill(0)
    yLine_myoglobinPS.fill(myoglobinPS_at_oppVal)
    yLine_hemoglobinPS.fill(hemoglobinPS_at_oppVal)
    
    yLine_oppValArray[yLine_oppValArray.length-1] = o_ppVal
    
    for (var indexVal = 0; indexVal < oPP.length; indexVal++) {
        myoglobinPercentSaturation[indexVal] = (( (oPP[indexVal]** 1) / ((kp_mb** 1)+(oPP[indexVal]** 1)) ) ) * 100;
        hemoglobinPercentSaturation[indexVal] = (( (oPP[indexVal]** n_hb) / ((kp_hb** n_hb)+(oPP[indexVal])** n_hb) ) ) * 100;
          }
   
    source.change.emit();
""")


# an idea is to get the min and max values of each curve, and make that many points btw them for plotting

kp_mb_slider.js_on_change('value', callback)
kp_hb_slider.js_on_change('value', callback)

#n_mb_slider.js_on_change('value', callback)
n_hb_slider.js_on_change('value', callback)
o_pp_fig_slider.js_on_change('value', callback)


genericCasePlot.legend.location= "bottom_right"
genericCasePlot.legend.title_text_font_style = "bold"
genericCasePlot.legend.title_text_font_size = "15pt"

layout = row(
    genericCasePlot,
    widgetbox(kp_mb_slider, kp_hb_slider,n_hb_slider,o_pp_fig_slider)
)

output_notebook()


show(layout)

Loading BokehJS ...